# 「BERTの動的量子化（ベータ版）」

【原題】(beta) Dynamic Quantization on BERT

【原著】[Jianyu Huang](https://github.com/jianyuh)

【査読】[Raghuraman Krishnamoorthi](https://github.com/raghuramank100)

【編著】[Jessica Lin](https://github.com/jlin27)

【元URL】https://pytorch.org/tutorials/intermediate/dynamic_quantization_bert_tutorial.html

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年2月6日

【チュトーリアル概要】

本チュートリアルでは、HuggingFace TransformersのBERTモデルに動的量子化を適用します。

**ヒント**

本チュートリアルを最大限活用するには、こちらの[バージョンのColab](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/dynamic_quantization_bert_tutorial.ipynb)を使用することを推奨します。

## 導入

本チュートリアルでは、[HuggingFace Transformers](https://github.com/huggingface/transformers)のBERTモデルに動的量子化を適用します。

BERTのような有名かつ最先端のモデルを、動的量子化済みのモデルへと変換する方法について、ひとつずつ解説します。



- BERT、あるいはトランスフォーマーによる双方向型埋め込み表現（Bidirectional Embedding Representations from Transformers）は、質問回答や文書分類などの多くの自然言語処理（NLP）タスクにおいて最先端の精度を達成している、新しい訓練済みの言語表現の方法の一つです。
  元の論文は[こちら](https://arxiv.org/pdf/1810.04805.pdf)で確認できます。

- PyTorchでサポートされている動的量子化は、重みや活性化について動的量子化を行い、float型のモデルを静的なint8型やfloat16型の量子化モデルへと変換します。
  
  活性化は重みがint8型に量子化される際に、（バッチ毎に）動的にint8型へと量子化されます。
  
  PyTorchでは、特定のモジュールの重みのみ量子化したものに置換し、量子化モデルを出力する[torch.quantization.quantize_dynamic API](https://pytorch.org/docs/stable/quantization.html#torch.quantization.quantize_dynamic)が存在します。

- 一般的な言語理解評価のベンチマーク[(GLUE)](https://gluebenchmark.com/)の[Microsoft Research Paraphrase Corpus (MRPC)タスク](https://www.microsoft.com/en-us/download/details.aspx?id=52398)について、精度と推論のパフォーマンスの結果を求めます。
  MRPC (Dolan、Brockett, 2005)は、オンラインのニュースソースから自動的に抽出した文章のペアのコーパスであり、ペアの文章が意味的に等価かどうか人間がアノテーションを付けています。
  MRPCのクラスは不均衡（68%の陽性、32%の陰性）であるため、一般的な慣習に従い、[F1スコア](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)を指標にします。
  
  なお、以下に示すように、MRPCは言語ペアの分類において一般的なNLPのタスクです。

<img src='https://pytorch.org/tutorials/_images/bert.png'　 width="400" ></img>

## 1. 準備

### 1.1 PyTorchとHuggingFaceのTransformersのインストール

本チュートリアルを始めるにあたって、[PyTorch](https://github.com/pytorch/pytorch/#installation)と[HuggingFaceのGithubのリポジトリ](https://github.com/huggingface/transformers#installation)にあるインストール方法に倣いましょう。
さらに、組み込み済みのF1スコアを算出する関数を利用するため、[scikit-learn](https://github.com/scikit-learn/scikit-learn)もインストールします。

In [ ]:
!pip install sklearn
!pip install transformers

     |████████████████████████████████| 1.8MB 5.6MB/s 
     |████████████████████████████████| 890kB 16.6MB/s 
     |████████████████████████████████| 3.2MB 26.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f9c7195f69375d43d5563efe3738c44005f7936733ed5cec7a255b7145f9a0c1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


なお、PyTorchのベータ版である機能を使用することになるため、最新バージョンのtorchとtorchvisionをインストールすることを推奨します。
最新のローカルでのインストール方法は[こちら](https://pytorch.org/get-started/locally/)で確認可能です。<br>
例えば、Mac上にインストールするには下記のコマンドを使用します。

In [ ]:
# !yes y | pip uninstall torch tochvision
# !yes y | pip install --pre torch -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html

### 1.2 必須モジュールのインポート

本ステップでは、チュートリアルを進めるにあたって必須であるPythonのモジュールをインポートします。

In [ ]:
from __future__ import absolute_import, division, print_function

import logging
import numpy as np
import os
import random
import sys
import time
import torch

from argparse import Namespace
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from tqdm import tqdm
from transformers import (BertConfig, BertForSequenceClassification, BertTokenizer,)
from transformers import glue_compute_metrics as compute_metrics
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors
from transformers import glue_convert_examples_to_features as convert_examples_to_features

# ログの準備
logger = logging.getLogger(__name__)
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.WARN)

logging.getLogger("transformers.modeling_utils").setLevel(
   logging.WARN)  # ログの削減

print(torch.__version__)

1.7.0+cu101


FP32とINT8との間で、単一スレッドでのパフォーマンスを比較するために、スレッド数を設定します。

なお本チュートリアルの最後では、適切な並列バックエンドを持つPyTorchを構築することで、別途スレッド数を設定できるようになります。


In [ ]:
torch.set_num_threads(1)
print(torch.__config__.parallel_info())

ATen/Parallel:
	at::get_num_threads() : 1
	at::get_num_interop_threads() : 1
OpenMP 201511 (a.k.a. OpenMP 4.5)
	omp_get_max_threads() : 1
Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
	mkl_get_max_threads() : 1
Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
std::thread::hardware_concurrency() : 2
Environment variables:
	OMP_NUM_THREADS : [not set]
	MKL_NUM_THREADS : [not set]
ATen parallel backend: OpenMP



### 1.3 ヘルパー関数について学ぶ

transformersのライブラリには組み込み済みのヘルパー関数が存在します。<br>
本チュートリアルでは、主に2つのヘルパー関数を使用します。

一つは文章のサンプルを特徴ベクトルに変換する関数であり、もう一つは予測結果のF1スコアを測定する関数です。



[glue_convert_examples_to_features](https://github.com/huggingface/transformers/blob/master/transformers/data/processors/glue.py)関数は、文章を入力特徴量に変換します。

- 入力系列をトークン化します。
- 始めの部分に[CLS]を挿入します。
- 初めの文章と2番目の文章との間、及び最後の部分に[SEP]を挿入します。
- トークンが最初の文章に属するか、2番目の文章に属するかを示すトークン型IDを生成します。

[glue_compute_metrics](https://github.com/huggingface/transformers/blob/master/transformers/data/processors/glue.py)関数は、適合率と再現率の加重平均と解釈できる、[F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)の指標を備えています。

F1スコアは最良の値で1を取り、最悪の場合は0を取ります。

なお、F1スコアに対する適合率と再現率の寄与は相対的に等しいものになっています。

- F1スコアの式は以下のとおりです。

$$
F1 = 2 * (\text{precision} * \text{recall}) / (\text{precision} + \text{recall})
$$

### 1.4 データセットのダウンロード

MRPCタスクを実行する前に、[こちらのスクリプト](https://gist.github.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e)を実行して[GLUEデータ](https://gluebenchmark.com/tasks)をダウンロードし、`glue_data`ディレクトリに解凍します。

In [ ]:
!python download_glue_data.py --data_dir='glue_data' --tasks='MRPC'

# 日本語版注釈：ここがうまく動作しない・・・

## 2. BERTモデルのファインチューン

言語表現を事前訓練し、様々なタスクに対して、最小限のタスク依存のパラメータを使用してディープな双方向表現をファインチューニングし、最先端の結果を達成する

という流れが、BERTのアイデアです。

本チュートリアルでは、MRPCタスクに存在する、意味的に等価な文章のペアを分類する事前訓練済みのBERTモデルをファインチューニングすることに専念します。

事前訓練済みのBERTモデル（HuggingFaceのtransformers内の`bert-base-uncased`モデル）を、MRPCタスクに対してファインチューニングするには、[こちらのサンプル](https://github.com/huggingface/transformers/tree/master/examples#mrpc)のコマンドにならいます。

In [ ]:
!export GLUE_DIR=./glue_data
!export TASK_NAME=MRPC
!export OUT_DIR=./$TASK_NAME/
!python ./run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name $TASK_NAME \
    --do_train \
    --do_eval \
    --do_lower_case \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --save_steps 100000 \
    --output_dir $OUT_DIR

MRPCタスクのためにファインチューン済みのBERTモデルを[こちら](https://download.pytorch.org/tutorial/MRPC.zip)で提供しています。

時間を節約するために、ローカルの`$OUT_DIR`フォルダにモデルのファイル（~400 MB）をダウンロードすることが可能です。

### 2.1 グローバルな設定

動的量子化の前後でファインチューンされたBERTモデルの評価を行うために、グローバルな設定を行います。

In [ ]:
configs = Namespace()

# ファインチューン済みモデルを出力するディレクトリ、$OUT_DIR
configs.output_dir = "./MRPC/"

# GLUEベンチマークのMRPCタスクのデータを格納するディレクトリ、$GLUE_DIR/$TASK_NAME
configs.data_dir = "./glue_data/MRPC"

# 事前訓練済みモデルのモデル名、またはパス
configs.model_name_or_path = "bert-base-uncased"
# 入力系列の最大長
configs.max_seq_length = 128

# GLUEタスクの準備
configs.task_name = "MRPC".lower()
configs.processor = processors[configs.task_name]()
configs.output_mode = output_modes[configs.task_name]
configs.label_list = configs.processor.get_labels()
configs.model_type = "bert".lower()
configs.do_lower_case = True

# デバイス、バッチサイズ、トポロジ（GPU数やマシンのランク）、及びキャッシュフラグを設定
configs.device = "cpu"
configs.per_gpu_eval_batch_size = 8
configs.n_gpu = 0
configs.local_rank = -1
configs.overwrite_cache = False


# 再現性のための乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
set_seed(42)


### 2.2 ファインチューン済みBERTモデルの読み込み

`configs.output_dir`からトークナイザ―とファインチューニング済みのBERT系列分類器モデル（FP32）を読み込みます。

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    configs.output_dir, do_lower_case=configs.do_lower_case)

model = BertForSequenceClassification.from_pretrained(configs.output_dir)
model.to(configs.device)

### 2.3 トークン化と評価を行う関数の定義

[Huggingface](https://github.com/huggingface/transformers/blob/master/examples/run_glue.py)からトークン化と評価を行う関数を再利用します。

In [ ]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

def evaluate(args, model, tokenizer, prefix=""):
    # MNLIの2つの評価(一致、不一致)を処理するためのループ 
    eval_task_names = ("mnli", "mnli-mm") if args.task_name == "mnli" else (args.task_name,)
    eval_outputs_dirs = (args.output_dir, args.output_dir + '-MM') if args.task_name == "mnli" else (args.output_dir,)

    results = {}
    for eval_task, eval_output_dir in zip(eval_task_names, eval_outputs_dirs):
        eval_dataset = load_and_cache_examples(args, eval_task, tokenizer, evaluate=True)

        if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(eval_output_dir)

        args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
        # DistributedSamplerはランダムにサンプリングする点に留意してください。
        eval_sampler = SequentialSampler(eval_dataset) if args.local_rank == -1 else DistributedSampler(eval_dataset)
        eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

        # マルチGPUの確認
        if args.n_gpu > 1:
            model = torch.nn.DataParallel(model)

        # 評価
        logger.info("***** Running evaluation {} *****".format(prefix))
        logger.info("  Num examples = %d", len(eval_dataset))
        logger.info("  Batch size = %d", args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            model.eval()
            batch = tuple(t.to(args.device) for t in batch)

            with torch.no_grad():
                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'labels':         batch[3]}
                if args.model_type != 'distilbert':
                    inputs['token_type_ids'] = batch[2] if args.model_type in ['bert', 'xlnet'] else None  # XLM、DistilBERT、そしてRoBERTaはsegment_idsを使用しません。
                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1
            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs['labels'].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        if args.output_mode == "classification":
            preds = np.argmax(preds, axis=1)
        elif args.output_mode == "regression":
            preds = np.squeeze(preds)
        result = compute_metrics(eval_task, preds, out_label_ids)
        results.update(result)

        output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results {} *****".format(prefix))
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))

    return results


def load_and_cache_examples(args, task, tokenizer, evaluate=False):
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  # 分散訓練内の最初のプロセスのみがデータセットを処理し、その他のプロセスはキャッシュを使用するように担保します。

    processor = processors[task]()
    output_mode = output_modes[task]
    # キャッシュ、またはデータセットのファイルからデータの特徴量を読み込み
    cached_features_file = os.path.join(args.data_dir, 'cached_{}_{}_{}_{}'.format(
        'dev' if evaluate else 'train',
        list(filter(None, args.model_name_or_path.split('/'))).pop(),
        str(args.max_seq_length),
        str(task)))
    if os.path.exists(cached_features_file) and not args.overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        label_list = processor.get_labels()
        if task in ['mnli', 'mnli-mm'] and args.model_type in ['roberta']:
            # ハック：RoBERTaの訓練済みモデルではラベルのインデックスが反転しています。
            label_list[1], label_list[2] = label_list[2], label_list[1]
        examples = processor.get_dev_examples(args.data_dir) if evaluate else processor.get_train_examples(args.data_dir)
        features = convert_examples_to_features(examples,
                                                tokenizer,
                                                label_list=label_list,
                                                max_length=args.max_seq_length,
                                                output_mode=output_mode,
                                                pad_on_left=bool(args.model_type in ['xlnet']),                 # xlnetでは左部にパディングを挿入します。
                                                pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                pad_token_segment_id=4 if args.model_type in ['xlnet'] else 0,
        )
        if args.local_rank in [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)

    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier()  # 分散訓練内の最初のプロセスのみがデータセットを処理し、その他のプロセスはキャッシュを使用するように担保します。

    # テンソルに変換し、データセットを構築します。
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
    return dataset



## 3. 動的量子化の適用

`torch.quantization.quantize_dynamic`をモデルに対して呼び出し、HuggingFaceのBERTモデルに動的量子化を適用します。

具体的には、以下の処理が行われるように引数を指定します。

- モデル内のtorch.nn.Linearモジュールが量子化される
- 重みが量子化されたint8型の値に変換される

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

### 3.1 モデルサイズの確認

まずはモデルのサイズを確認してみましょう。
モデルサイズが大幅に削減されていることが観測できます。(FP32 総サイズ: 438 MB; INT8 総サイズ: 181 MB)

In [ ]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

本チュートリアルで使用されているBERTモデル (`bert-base-uncased`) は、V=30522のボキャブラリーサイズです。

つまり、埋め込みサイズが768であれば、埋め込みテーブルの総サイズはおよそ 4 (Bytes/FP32) * 30522 * 768 = 90 MB になります。

したがって、量子化の効果で非埋め込みテーブルの部分のモデルサイズは、350 MB (FP32モデル)から90 MB (INT8モデル)に削減されています。

### 3.2 推論精度と時間の評価

次に、元のFP32型モデルと動的量子化後のINT8型モデルの間で、推論時間と精度の評価を行います。

In [ ]:
def time_model_evaluation(model, configs, tokenizer):
    eval_start_time = time.time()
    result = evaluate(configs, model, tokenizer, prefix="")
    eval_end_time = time.time()
    eval_duration_time = eval_end_time - eval_start_time
    print(result)
    print("Evaluate total time (seconds): {0:.1f}".format(eval_duration_time))

# 元のFP32型のBERTモデルを評価
time_model_evaluation(model, configs, tokenizer)

# 動的量子化後のINT8型のBERTモデルを評価
time_model_evaluation(quantized_model, configs, tokenizer)

上記のコードを量子化せずにMacBook Pro上で実行した場合、（MRPCデータセット内の全408個のサンプルに対しての）推論には約160秒必要ですが、量子化を行った場合は約90秒しかかかりません。

MacBook Proで量子化済みBERTモデルを用いた推論を実行した結果を以下にまとめます。

<code>

    | Prec | F1 score | Model Size | 1 thread | 4 threads |
    | FP32 |  0.9019  |   438 MB   | 160 sec  | 85 sec    |
    | INT8 |  0.902   |   181 MB   |  90 sec  | 46 sec    |

</code>

MRPCタスクについてファインチューニングされたBERTモデルに対して訓練後に動的量子化を行った場合、0.6%のF1スコアが得られました。

なお比較として、[最近の論文](https://arxiv.org/pdf/1910.06188.pdf)(表1)では、訓練後に動的量子化を適用することで0.8788、量子化を考慮した訓練を行うことで0.8956のスコアを達成しています。

主な違いは、前記の論文は対称的な量子化のみサポートしている一方で、PyTorch では非対称的な量子化をサポートしている点です。

なお、本チュートリアルでは単一スレッドでの比較を行うためにスレッド数を1に設定している点に留意してください。

これらの量子化INT8型の演算子の処理の並列化もサポートしています。
ユーザーは`torch.set_num_threads(N)`（Nは処理中に並列化するスレッド数）により、マルチスレッドを設定することが可能です。

ただし、処理の並列化サポートを有効化するために必要な補足事項として、PyTorchをOpenMP、Native、またはTBBといった適切な[バックエンド](https://pytorch.org/docs/stable/notes/cpu_threading_torchscript_inference.html#build-options)でビルドする必要があります。

`torch.__config__.parallel_info()`を使用し、並列化の設定を確認することが可能です。

ちなみに、同一のMacBook Proで並列化にNativeバックエンドを使用したPyTorchでは、MRPCのデータセットの評価の処理を約46秒で行えました。

### 3.3 量子化モデルのシリアル化

モデルのトレース後に`torch.jit.save`を使用することで、量子化モデルをシリアル化し、保存することが可能です。

In [ ]:
input_ids = ids_tensor([8, 128], 2)
token_type_ids = ids_tensor([8, 128], 2)
attention_mask = ids_tensor([8, 128], vocab_size=2)
dummy_input = (input_ids, attention_mask, token_type_ids)
traced_model = torch.jit.trace(quantized_model, dummy_input)
torch.jit.save(traced_model, "bert_traced_eager_quant.pt")

量子化モデルを読み込むには、`torch.jit.load`が使用可能です。

In [ ]:
loaded_quantized_model = torch.jit.load("bert_traced_eager_quant.pt")

## 結論

本チュートリアルでは、BERTのような有名で最先端のNLPモデルを動的に量子化されたモデルへと変換する方法を解説しました。

動的量子化は、精度への影響を限定的なものとしつつ、モデルのサイズを削減します。

本チュートリアルに目を通していただき、ありがとうございます。

どんなフィードバックでも歓迎するので、もし何かございましたら、[こちら](https://github.com/pytorch/pytorch/issues)にissueを作成してください。

（日本語訳注：日本語版チュートリアルに関連する内容であれば、是非[こちら](https://github.com/YutaroOgawa/pytorch_tutorials_jp)にissueの登録をお願いいたします。）

## 引用

[1] J.Devlin、 M. Chang、 K. Lee、 K. Toutanova, [BERT: 言語理解のための双方向ディープトランスフォーマーの事前訓練(2018)](https://arxiv.org/pdf/1810.04805.pdf).

[2] [HuggingFace Transformers](https://github.com/huggingface/transformers).

[3] O. Zafrir、 G. Boudoukh、 P. Izsak、M. Wasserblat (2019). [Q8BERT: 量子化8bit BERT](https://arxiv.org/pdf/1910.06188.pdf).